In [1]:
import os
import pandas as pd
os.chdir('../..')

In [2]:
os.getcwd()

'/Users/lukestrange/Code/true-north'

# Net Zero

In [3]:
TDC_DIR = 'working/datacity'
NETZERO_DIR = os.path.join(TDC_DIR, 'netzero')
OUTDIR = 'src/themes/sustainable-growth/netzero/_data'

In [4]:
data = pd.read_csv(os.path.join(NETZERO_DIR, 'All_UK_companies-Company_size_by_employees.csv'))
data = data[data['Company size'] != '250+ employees']
data.to_csv(os.path.join(OUTDIR, 'company_size_by_employees.csv'))

In [5]:
data = pd.read_csv(os.path.join(NETZERO_DIR, 'All_UK_companies-Employees_by_year.csv'))
data.rename(columns={"Measured total (GBP)": "measured_total", "Projected total (GBP)": "projected_total"}, inplace=True)
data.to_csv(os.path.join(OUTDIR, 'employees_by_year.csv'), index=False)

In [6]:
data = pd.read_csv(os.path.join(NETZERO_DIR, 'All_UK_companies-Turnover_by_year.csv'))
data.to_csv(os.path.join(OUTDIR, 'turnover_by_year.csv'), index=False)

In [7]:
from pipelines.util import *

### Adding geography codes to datacity data

In [8]:
def get_relative_paths(dir, keyword):
    f_paths = []
    names = []
    for root, _, files in os.walk(dir):
        for file in files:
            if file == '.gitignore':
                continue
            if f'{keyword}' in file:
                f_paths.append(os.path.join(root, file))
                names.append(file.split("-")[1])
    return f_paths, names

In [9]:
def get_geo_codes(INDIR, OUTDIR):
    paths, names = get_relative_paths(INDIR, 'local_authority')
    print(names, paths)
    # outs = [os.path.join('src/themes/sustainable-growth/netzero/_data/', f'{name}.csv') for name in ['business_counts', 'employee_counts', 'turnover']]

    codes = pd.read_csv('metadata/LAD23_lookup.csv', usecols=['LAD23NM', 'LAD23CD'])

    for path, name in zip(paths, names):
        data = pd.read_csv(f'{path}')
        data.rename(columns={'Local authority': 'LAD23NM'}, inplace=True)
        merged = data.merge(codes, how='inner', on='LAD23NM') 
        merged.to_csv(os.path.join(OUTDIR,f'{name}'), index=False)
    return

In [10]:
get_geo_codes('working/datacity/netzero', 'src/themes/sustainable-growth/netzero/_data/')
get_geo_codes('working/datacity/owner-managed', 'src/themes/true-north/owner-managed/_data/')

['Business_counts_by_local_authority.csv', 'Turnovers_by_local_authority.csv', 'Employees_by_local_authority.csv'] ['working/datacity/netzero/All_UK_companies-Business_counts_by_local_authority.csv', 'working/datacity/netzero/All_UK_companies-Turnovers_by_local_authority.csv', 'working/datacity/netzero/All_UK_companies-Employees_by_local_authority.csv']
['Business_counts_by_local_authority.csv', 'Turnovers_by_local_authority.csv', 'Employees_by_local_authority.csv'] ['working/datacity/owner-managed/All_UK_companies-Business_counts_by_local_authority.csv', 'working/datacity/owner-managed/All_UK_companies-Turnovers_by_local_authority.csv', 'working/datacity/owner-managed/All_UK_companies-Employees_by_local_authority.csv']


### Get top 10 sectors for owner managed rtic sector data

In [11]:
OUTDIR = 'src/themes/true-north/owner-managed/_data'
paths, names = get_relative_paths('working/datacity/owner-managed', 'RTIC')
for path, name in zip(paths, names):
    data = pd.read_csv(path)
    data.sort_values(by='Count', inplace=True, ascending=False)
    data = data.head(10)
    data.to_csv(os.path.join(OUTDIR, f'{name}'), index=False)